In [1]:
from math import *
from sympy import *
from vpython import *

#1 Given the value for precision control variable
eps = 0.01
# assign symbols in sympy to solve system of equation
x,y,z=symbols('x y z')

#2 Functions for main program
first_layer = lambda n: float(sin(pi/n)/(1-sin(pi/n)))
f = lambda p,q: sqrt((p+q)**2-q**2)
select = lambda c1, c2, c3, c4: c3 if (c2[0]-c1[0])*(c3[1]-c1[1])-(c2[1]-c1[1])*(c3[0]-c1[0])>0 else c4
oneangle = lambda r, r0, r1: acos((r**2+r*(r0+r1)-r0*r1)/((r+r0)*(r+r1)))
fixangle = lambda r: 2*oneangle(r,r,1)
totalangle = lambda m,r, L: fixangle(r)+2*oneangle(r,r,L[0])+2*radiiangle(m,r,L)
radiiangle = lambda m,r,L: sum([oneangle(r,L[i],L[i+1]) for i in range(len(L)-1)]) + (oneangle(r,L[-1],L[-1])/2 if m%2 else 0)
def circlelist(m,L,c1,c2,c3):
    if m == 0: 
        return L
    else:
        c=apollonius(c1,c2,c3)
        L = L+[c]
        c=(c[0],c[1],-c[2])
        return circlelist(m-1, L, c1, c, c3) 

#3 Given 3 circles, find the circle (tangent to 3 circles) we want
def apollonius(c1, c2, c3): 
    x1, y1, z1=c1;    x2, y2, z2=c2;     x3, y3, z3=c3          
    eqn1, eqn2, eqn3  = (x-x1)**2+(y-y1)**2-(z-z1)**2, (x-x2)**2+(y-y2)**2-(z-z2)**2, (x-x3)**2+(y-y3)**2-(z-z3)**2
    eqn13, eqn12 = delsmall(simplify(eqn1-eqn3)), delsmall(simplify(eqn1-eqn2))    
    if y1==y2==y3:
        L=solve([eqn13, eqn12], [x,z])
        xy,zy=float(L[x]), float(L[z])
        eqnlast = eqn1.subs([(x,xy),(z,zy)])    # f(z) is a degree 2 equation
        L=solve(eqnlast, y)
        c3,c4 = (xy, float(L[0]), zy), (xy, float(L[1]), zy)        
    elif x1==x2==x3:
        L=solve([eqn13, eqn12], [y,z])
        yx,zx=float(L[y]), float(L[z])
        eqnlast = eqn1.subs([(y,yx),(z,zx)])    # f(z) is a degree 2 equation
        L=solve(eqnlast, x) 
        c3,c4 = (float(L[0]), yx, zx),(float(L[1]), yx, zx)
    else: 
        L=solve([eqn13, eqn12],[x,y])
        xz,yz=L[x], L[y]
        eqnlast = simplify(eqn1.subs([(x,xz),(y,yz)]))    # f(z) is a degree 2 equation
        L=solve(eqnlast, z)         # solve for y
        z1, z2 = float(L[0]), float(L[1])
        x1, x2 = float(xz.subs([(z,z1)])), float(xz.subs([(z,z2)]))
        y1, y2 = float(yz.subs([(z,z1)])), float(yz.subs([(z,z2)]))
        c3, c4 = (x1,y1,z1),(x2,y2,z2) 
    return select(c1,c2,c3,c4)
def delsmall(eq): # Delete terms who has small coefficient
    p = Poly(eq)
    for i in [abs(i) for i in p.coeffs() if abs(i) < eps]:
        p = p.subs(i,0)
    return p  

#4 The following are functions for display the result
def draw(n,r):
    ring(pos=vector(0,0,0), axis=vector(0,0,1), radius=1, thickness=0.02, color=color.red)
    for theta in arange(0, 2*pi, 2*pi/n):
        ring(pos=vector((1+r)*cos(theta),(1+r)*sin(theta),0), axis=vector(0,0,1), radius=r, thickness=0.02, color=color.red)
def displayone(c,n):
    x,y,r=c
    l = sqrt(x**2+y**2)
    start, step = atan(y/x), 2*pi/n
    for theta in arange(start, 2*pi, step):
        ring(pos=vector(l*cos(theta), l*sin(theta), 0), axis=vector(0,0,1), radius=c[2], thickness=0.01, color=color.blue)
def show(r,rr,clist,n,l):
    scene.background=vec(0.9,0.9,0.9)
    draw(n,r)
    for i in clist:
        displayone(i,n)
    ring(pos=vector(0,0,0), axis=vector(0,0,1), radius=l+rr, thickness=0.01, color=color.green)

#5 Given n,m, main(n,m) display the BC[1,n,nxm] problem
def main(n,m):    
    r = first_layer(n)
    rr = 2*r
    rmin, rmax,  l = 0, 2*r, f(1,r)+f(rr,r)  
    c1,c2,c3 = ((1+r),0,-r),(float(l*cos(-pi/n)), float(l*sin(-pi/n)), -rr),(0,0,l+rr)
    clist = circlelist(m//2,[c2],c1,c2,c3)
    rlist=[abs(i[2]) for i in clist]
    sa = totalangle(m,r,rlist)
    while abs(sa-2*pi)>eps:
        rr = (rmin+rmax)/2
        l = f(1,r)+f(rr,r)   
        c1, c2, c3 = ((1+r),0,-r), (l*cos(-pi/n), l*sin(-pi/n), -rr), (0,0,l+rr)
        clist = circlelist(m//2,[c2],c1,c2,c3)
        rlist = [abs(i[2]) for i in clist]
        c = apollonius(c1,c2,c3) # ?
        sa = totalangle(m,r,rlist)
        if sa>2*pi:
            rmax = rr
        else:
            rmin = rr
    clist += [(i[0], -i[1], i[2]) for i in clist[1:][::-1]] if m%2 else [(i[0], -i[1], i[2]) for i in clist[1:-1][::-1]]
    show(r,rr,clist,n,l) 

main(7,7)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>